In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation
from tensorflow.keras.layers import add, Add

def identity_block(input_tensor, middle_kernel_size, filters, stage, block):
  filter1, filter2, filter3 = filters

  conv_name_base = f'res{stage}{block}_branch'
  bn_name_base = f'bn{stage}{block}_branch'

  x = Conv2D(filters=filter1, kernel_size=(1, 1), kernel_initializer='he_normal', name=f'{conv_name_base}2a')(input_tensor)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2a')(x)
  x = Activation('relu')(x)

  x = Conv2D(filters=filter2, kernel_size=middle_kernel_size, padding='same', kernel_initializer='he_normal', name=f'{conv_name_base}2b')(x)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2b')(x)
  x = Activation('relu')(x)

  x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=f'{conv_name_base}2c')(x)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2c')(x)

  x = Add()([input_tensor, x])
  # x = add([input_tensor, x])

  x = Activation('relu')(x)

  return x

In [3]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_tensor = Input(shape=(56, 56, 256), name='test_input')
filters = [64, 64, 256]
kernel_size = (3, 3)
stage = 2
block = 'a'

output = identity_block(input_tensor, kernel_size, filters, stage, block)
identity_layers = Model(inputs=input_tensor, outputs=output)

identity_layers.summary()

2022-09-30 07:12:38.057243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 07:12:38.148729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 07:12:38.149534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-30 07:12:38.151268: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
test_input (InputLayer)         [(None, 56, 56, 256) 0                                            
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 56, 56, 64)   16448       test_input[0][0]                 
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation (Activation)         (None, 56, 56, 64)   0           bn2a_branch2a[0][0]              
______________________________________________________________________________________________

In [4]:
input_tensor = Input(shape=(56, 56, 256), name='test_input')
x = identity_block(input_tensor, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='a')
x = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='b')
output = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='c')

identity_layers = Model(inputs=input_tensor, outputs=output)

identity_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
test_input (InputLayer)         [(None, 56, 56, 256) 0                                            
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 56, 56, 64)   16448       test_input[0][0]                 
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 56, 56, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 56, 56, 64)   0           bn2a_branch2a[0][0]              
____________________________________________________________________________________________

In [5]:
def conv_block(input_tensor, middle_kernel_size, filters, stage, block, strides=(2, 2)):
  filter1, filter2, filter3 = filters

  conv_name_base = f'res{stage}{block}_branch'
  bn_name_base = f'bn{stage}{block}_branch'

  x = Conv2D(filters=filter1, kernel_size=(1, 1), strides=strides, kernel_initializer='he_normal', name=f'{conv_name_base}2a')(input_tensor)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2a')(x)
  x = Activation('relu')(x)

  x = Conv2D(filters=filter2, kernel_size=middle_kernel_size, padding='same', kernel_initializer='he_normal', name=f'{conv_name_base}2b')(x)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2b')(x)
  x = Activation('relu')(x)

  x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=f'{conv_name_base}2c')(x)
  x = BatchNormalization(axis=3, name=f'{bn_name_base}2c')(x)

  shortcut = Conv2D(filter3, (1, 1), strides=strides, kernel_initializer='he_normal', name=f'{conv_name_base}1')(input_tensor)
  shortcut = BatchNormalization(axis=3, name=f'{bn_name_base}1')(shortcut)
  
  x = Add()([shortcut, x])
  # x = add([shortcut, x])

  x = Activation('relu')(x)

  return x

In [6]:
input_tensor = Input(shape=(56, 56, 256), name='test_input')
x = conv_block(input_tensor, middle_kernel_size=3, filters=[64, 64, 256], strides=2, stage=2, block='a')
x = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='b')
output = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='c')
identity_layers = Model(inputs=input_tensor, outputs=output)

identity_layers.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
test_input (InputLayer)         [(None, 56, 56, 256) 0                                            
__________________________________________________________________________________________________
res2a_branch2a (Conv2D)         (None, 28, 28, 64)   16448       test_input[0][0]                 
__________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizati (None, 28, 28, 64)   256         res2a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 28, 28, 64)   0           bn2a_branch2a[0][0]              
____________________________________________________________________________________________

In [7]:
from tensorflow.keras.layers import ZeroPadding2D, MaxPooling2D

def do_first_conv(input_tensor):
  x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_tensor)
  x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid', kernel_initializer='he_normal', name='conv')(x)
  x = BatchNormalization(axis=3, name='bn_conv1')(x)
  x = Activation('relu')(x)
  x = ZeroPadding2D(padding=(1, 1), name='pool_1_pad')(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  return x

In [8]:
input_tensor = Input(shape=(224, 224, 3))
output = do_first_conv(input_tensor)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 230, 230, 3)       0         
_________________________________________________________________
conv (Conv2D)                (None, 112, 112, 64)      9472      
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 112, 112, 64)      256       
_________________________________________________________________
activation_21 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
pool_1_pad (ZeroPadding2D)   (None, 114, 114, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 64)        0   

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

def create_resnet(in_shape=(224, 224, 3), n_classes=10):
  input_tensor = Input(shape=in_shape)

  x = do_first_conv(input_tensor)

  x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
  x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
  x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

  x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
  x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

  x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
  x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

  x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
  x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
  x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

  x = GlobalAveragePooling2D(name='avg_pool')(x)
  x = Dropout(rate=0.5)(x)
  x = Dense(200, activation='relu', name='fc_01')(x)
  x = Dropout(rate=0.5)(x)
  output = Dense(n_classes, activation='softmax', name='fc_final')(x)

  model = Model(inputs=input_tensor, outputs=output, name='resnet50')
  model.summary()

  return model

In [10]:
model =  create_resnet(in_shape=(224,224,3), n_classes=10)

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv (Conv2D)                   (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv[0][0]                       
___________________________________________________________________________________________

In [11]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [12]:
import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

from tensorflow.keras.utils import Sequence
import cv2
import sklearn

# 입력 인자 images_array labels는 모두 numpy array로 들어옴. 
# 인자로 입력되는 images_array는 전체 32x32 image array임. 
class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        '''
        파라미터 설명
        images_array: 원본 32x32 만큼의 image 배열값. 
        labels: 해당 image의 label들
        batch_size: __getitem__(self, index) 호출 시 마다 가져올 데이터 batch 건수
        augmentor: albumentations 객체
        shuffle: 학습 데이터의 경우 epoch 종료시마다 데이터를 섞을지 여부
        '''
        # 객체 생성 인자로 들어온 값을 객체 내부 변수로 할당. 
        # 인자로 입력되는 images_array는 전체 32x32 image array임.
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        # train data의 경우 
        self.shuffle = shuffle
        if self.shuffle:
            # 객체 생성시에 한번 데이터를 섞음. 
            #self.on_epoch_end()
            pass
    
    # Sequence를 상속받은 Dataset은 batch_size 단위로 입력된 데이터를 처리함. 
    # __len__()은 전체 데이터 건수가 주어졌을 때 batch_size단위로 몇번 데이터를 반환하는지 나타남
    def __len__(self):
        # batch_size단위로 데이터를 몇번 가져와야하는지 계산하기 위해 전체 데이터 건수를 batch_size로 나누되, 정수로 정확히 나눠지지 않을 경우 1회를 더한다. 
        return int(np.ceil(len(self.labels) / self.batch_size))
    
    # batch_size 단위로 image_array, label_array 데이터를 가져와서 변환한 뒤 다시 반환함
    # 인자로 몇번째 batch 인지를 나타내는 index를 입력하면 해당 순서에 해당하는 batch_size 만큼의 데이타를 가공하여 반환
    # batch_size 갯수만큼 변환된 image_array와 label_array 반환. 
    def __getitem__(self, index):
        # index는 몇번째 batch인지를 나타냄. 
        # batch_size만큼 순차적으로 데이터를 가져오려면 array에서 index*self.batch_size:(index+1)*self.batch_size 만큼의 연속 데이터를 가져오면 됨
        # 32x32 image array를 self.batch_size만큼 가져옴. 
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        # 만일 객체 생성 인자로 albumentation으로 만든 augmentor가 주어진다면 아래와 같이 augmentor를 이용하여 image 변환
        # albumentations은 개별 image만 변환할 수 있으므로 batch_size만큼 할당된 image_name_batch를 한 건씩 iteration하면서 변환 수행. 
        # 변환된 image 배열값을 담을 image_batch 선언. image_batch 배열은 float32 로 설정. 
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        # batch_size에 담긴 건수만큼 iteration 하면서 opencv image load -> image augmentation 변환(augmentor가 not None일 경우)-> image_batch에 담음. 
        for image_index in range(images_fetch.shape[0]):
            #image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            # 원본 image를 IMAGE_SIZE x IMAGE_SIZE 크기로 변환
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
            # 만약 augmentor가 주어졌다면 이를 적용. 
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
            # 만약 scaling 함수가 입력되었다면 이를 적용하여 scaling 수행. 
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            # image_batch에 순차적으로 변환된 image를 담음.               
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    # epoch가 한번 수행이 완료 될 때마다 모델의 fit()에서 호출됨. 
    def on_epoch_end(self):
        if(self.shuffle):
            #print('epoch end')
            # 원본 image배열과 label를 쌍을 맞춰서 섞어준다. scikt learn의 utils.shuffle에서 해당 기능 제공
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass

In [13]:
# CIFAR10 데이터 재 로딩 및 OHE 전처리 적용하여 학습/검증/데이터 세트 생성. 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

170508288/170498071 [==============================] - 7s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3) (10000, 10)


In [14]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=resnet_preprocess)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)

In [ ]:
resnet_model = create_resnet(in_shape=(128, 128, 3), n_classes=10)

resnet_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 5번 iteration내에 validation loss가 향상되지 않으면 learning rate을 기존 learning rate * 0.2로 줄임.  
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

history = resnet_model.fit(tr_ds, epochs=30, 
                    #steps_per_epoch=int(np.ceil(tr_images.shape[0]/BATCH_SIZE)),
                    validation_data=val_ds, 
                    #validation_steps=int(np.ceil(val_images.shape[0]/BATCH_SIZE)), 
                    callbacks=[rlr_cb, ely_cb]
                   )

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv (Conv2D)                   (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv[0][0]                       
___________________________________________________________________________________________

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-09-30 07:12:52.777189: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-30 07:12:59.896584: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


625/625 [==============================] - 105s 147ms/step - loss: 1.9911 - accuracy: 0.2496 - val_loss: 11.3963 - val_accuracy: 0.1521
Epoch 2/30
625/625 [==============================] - 90s 144ms/step - loss: 1.5986 - accuracy: 0.4046 - val_loss: 1.3201 - val_accuracy: 0.5133
Epoch 3/30
625/625 [==============================] - 90s 144ms/step - loss: 1.3926 - accuracy: 0.4989 - val_loss: 1.5291 - val_accuracy: 0.4804
Epoch 4/30
625/625 [==============================] - 90s 144ms/step - loss: 1.3020 - accuracy: 0.5352 - val_loss: 2.3084 - val_accuracy: 0.0963
Epoch 5/30
625/625 [==============================] - 93s 149ms/step - loss: 1.2139 - accuracy: 0.5705 - val_loss: 2.3582 - val_accuracy: 0.3887

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 6/30
625/625 [==============================] - 90s 145ms/step - loss: 0.9933 - accuracy: 0.6539 - val_loss: 0.9628 - val_accuracy: 0.6607
Epoch 7/30
625/625 [==============================] - 90s

In [ ]:
test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)
resnet_model.evaluate(test_ds)